## Runnables Basics - RunnableSequence, Runnable, RunnablePassthrough

In [2]:
%load_ext dotenv
%dotenv

In [3]:
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import ChatOpenAI

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [4]:
llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0,
    seed=108
)

In [5]:
chat_template_tools = ChatPromptTemplate.from_template(template='''
I am planning to start a career in {job title}, Can you tell me what are most important tools I need to master?

Display the output as a list of tool only.
''')

In [6]:
chat_template_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='\nI am planning to start a career in {job title}, Can you tell me what are most important tools I need to master?\n\nDisplay the output as a list of tool only.\n'), additional_kwargs={})])

In [7]:
chat_template_mastery = ChatPromptTemplate.from_template(template='''
Below are the list of tools, could you devise a strategy that I can follow to master each tool ?

{tools}
''')

In [8]:
chat_template_mastery

ChatPromptTemplate(input_variables=['tools'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tools'], input_types={}, partial_variables={}, template='\nBelow are the list of tools, could you devise a strategy that I can follow to master each tool ?\n\n{tools}\n'), additional_kwargs={})])

In [9]:
str_output = StrOutputParser()

In [10]:
chain_tools = chat_template_tools | llm | str_output

In [11]:
print(chain_tools.invoke({'job title': 'AI Engineer'}))

1. Python
2. TensorFlow
3. PyTorch
4. Keras
5. Scikit-learn
6. Jupyter Notebook
7. Pandas
8. NumPy
9. Matplotlib
10. OpenCV
11. Apache Spark
12. Docker
13. Git
14. SQL
15. Cloud Platforms (AWS, Google Cloud, Azure)


In [12]:
chain_mastery = chat_template_mastery | llm | str_output

In [13]:
print(chain_mastery.invoke({'tools':'''
1. Python
2. TensorFlow
3. PyTorch
4. Keras
5. Scikit-learn
6. Jupyter Notebook
7. Pandas
8. NumPy
9. Matplotlib
10. OpenCV
11. Apache Spark
12. Docker
13. Git
14. SQL
15. Cloud Platforms (AWS, Google Cloud, Azure)
'''}))

Mastering a diverse set of tools like the ones you've listed requires a structured approach. Below is a strategy that you can follow to effectively learn and master each tool. This strategy is divided into phases: foundational knowledge, practical application, project development, and continuous learning.

### Phase 1: Foundational Knowledge

1. **Python**
   - **Resources**: Online courses (Codecademy, Coursera, edX), books (Automate the Boring Stuff with Python).
   - **Practice**: Solve problems on platforms like LeetCode or HackerRank.

2. **SQL**
   - **Resources**: Online courses (SQL for Data Science on Coursera), books (SQL in 10 Minutes).
   - **Practice**: Use platforms like Mode Analytics or SQLZoo to practice queries.

3. **Git**
   - **Resources**: Git documentation, online courses (Git & GitHub on Coursera).
   - **Practice**: Create a GitHub account and start version controlling your projects.

### Phase 2: Data Manipulation and Analysis

4. **Pandas**
   - **Resources**

In [14]:
#Time to combine both the chains using RunnablePassthrough class to pass the output of one Runnable to another without modifying the input

chain_tools_passthrough = chat_template_tools | llm | str_output | {'tools':RunnablePassthrough()}
chain_mastery_stream = chat_template_mastery | llm

chain_combined = chain_tools_passthrough | chain_mastery_stream

In [15]:
ai_stream = chain_combined.stream({'job title':'Data Scientist'})

In [16]:
ai_stream

<generator object RunnableSequence.stream at 0x109fdec50>

In [17]:
for i in ai_stream:
    print(i.content, end='')

Mastering a set of tools like the ones you've listed requires a structured approach that combines theory, practice, and real-world application. Here’s a strategy you can follow for each tool:

### General Strategy
1. **Set Clear Goals**: Define what you want to achieve with each tool. For example, do you want to use Python for data analysis, machine learning, or web development?
2. **Structured Learning**: Use online courses, books, and tutorials. Websites like Coursera, edX, Udacity, and DataCamp offer structured courses.
3. **Hands-On Practice**: Implement projects or exercises that require you to use the tool. Websites like Kaggle provide datasets and competitions to practice.
4. **Join Communities**: Engage with communities on platforms like Stack Overflow, GitHub, or Reddit to ask questions and share knowledge.
5. **Build a Portfolio**: Document your projects and learning journey on platforms like GitHub or a personal blog.
6. **Iterate and Reflect**: Regularly review what you’ve 

In [18]:
# Chumma just wanted to see what the output of runnablepassthrough part was
print(chain_tools_passthrough.invoke({'job title':'Data Scientist'}))

{'tools': '1. Python\n2. R\n3. SQL\n4. Pandas\n5. NumPy\n6. Scikit-learn\n7. TensorFlow\n8. Keras\n9. Matplotlib\n10. Seaborn\n11. Jupyter Notebook\n12. Apache Spark\n13. Tableau\n14. Power BI\n15. Git'}


In [45]:
chain_combined.get_graph().print_ascii()

    +-------------+    
    | PromptInput |    
    +-------------+    
           *           
           *           
           *           
+--------------------+ 
| ChatPromptTemplate | 
+--------------------+ 
           *           
           *           
           *           
    +------------+     
    | ChatOpenAI |     
    +------------+     
           *           
           *           
           *           
 +------------------+  
 | ChatOpenAIOutput |  
 +------------------+  
           *           
           *           
           *           
  +-----------------+  
  | StrOutputParser |  
  +-----------------+  
           *           
           *           
           *           
    +-------------+    
    | Passthrough |    
    +-------------+    
           *           
           *           
           *           
+--------------------+ 
| ChatPromptTemplate | 
+--------------------+ 
           *           
           *           
           *    